<a href="https://colab.research.google.com/github/Mohammad-Moradi1/CNN-Pruning/blob/main/def_Extract_prunable_layers_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

این تابع وظیفه استخراج اطلاعات لایه‌های قابل هرس
(prunable)
را از یک مدل
PyTorch
 بر عهده دارد. این اطلاعات شامل لایه‌های قابل هرس، ابعاد خروجی آنها، و توزیع
 فیلترها است.
 ورودی‌ها
model: nn.Module
مدل PyTorch که لایه‌های قابل هرس از آن استخراج خواهند شد.

skip_layer_index: List
لیستی از ایندکس‌های لایه‌هایی که باید در فرآیند هرس نادیده گرفته شوند.

خروجی‌ها
filter_distribution: Tensor
توزیع نسبی ابعاد خروجی هر لایه قابل هرس به‌صورت یک تنسور PyTorch.

total_output_dim: int
مجموع ابعاد خروجی تمام لایه‌های قابل هرس.

prunable_layers: List
لیستی از اشیاء لایه‌های قابل هرس.

In [ ]:
import torch
from torch import nn, Tensor
from typing import Tuple, List, Dict
import queue



@torch.no_grad()
def extract_prunable_layers_info(model: nn.Module, skip_layer_index: List) -> Tuple[Tensor, int, List]:
    """ Extracts prunable layer information from a given neural network model """
    prunable_layers = []
    output_dims = []

    def recursive_extract_prunable_layers_info(module: nn.Module):
        """ Recursively extracts prunable layers from a module """
        children = list(module.children())
        for child in children:
            if isinstance(child, PRUNABLE_LAYERS):
                prunable_layers.append(child)
                if isinstance(child, CONV_LAYERS):
                    output_dims.append(child.out_channels)
                elif isinstance(child, nn.Linear):
                    output_dims.append(child.out_features)
            recursive_extract_prunable_layers_info(child)

    recursive_extract_prunable_layers_info(model)

    # skip the ouput layer as its out dim should equal to class num and can not be pruned
    del prunable_layers[-1]
    del output_dims[-1]

    prunable_layers = [item for idx, item in enumerate(prunable_layers) if idx not in skip_layer_index]
    output_dims = [item for idx, item in enumerate(output_dims) if idx not in skip_layer_index]

    total_output_dim = sum(output_dims)
    filter_distribution = [dim / total_output_dim for dim in output_dims]

    return torch.tensor(filter_distribution), total_output_dim, prunable_layers



کد مثال استفاده

In [ ]:
import torch
import torch.nn as nn
from typing import List, Tuple

# تعریف PRUNABLE_LAYERS و CONV_LAYERS
PRUNABLE_LAYERS = (nn.Conv2d, nn.Linear)
CONV_LAYERS = (nn.Conv2d,)

# یک مدل ساده با لایه‌های مختلف
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  # 3 -> 16
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)  # 16 -> 32
        self.fc1 = nn.Linear(32 * 8 * 8, 128)  # Fully connected layer
        self.fc2 = nn.Linear(128, 10)  # Output layer (10 classes)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

# مدل را تعریف کنید
model = SimpleModel()

# ایندکس لایه‌هایی که باید نادیده گرفته شوند (اینجا خالی است)
skip_layer_index = []

# فراخوانی تابع
filter_distribution, total_output_dim, prunable_layers = extract_prunable_layers_info(model, skip_layer_index)

# چاپ نتایج
print("Filter Distribution:", filter_distribution)
print("Total Output Dimension:", total_output_dim)
print("Prunable Layers:")
for i, layer in enumerate(prunable_layers):
    print(f"  Layer {i+1}: {layer}")